In [ ]:
#Task 1: Clean & preprocess the telco dataset, engineer features, and define a robust cross-validation strategy.
pip install shap
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# 1) Create a synthetic telco-style dataset (structure matches typical churn dataset)
def make_synthetic_telco(n=7043, random_state=42):
    rng = np.random.RandomState(random_state)
    tenure = rng.randint(0, 72, size=n)
    monthly = np.round(rng.uniform(18, 130, size=n), 2)
    total = np.round(np.maximum(tenure * monthly + rng.normal(0, 50, size=n), 0), 2)
    senior = rng.choice([0,1], size=n, p=[0.85,0.15])
    phone = rng.choice([0,1], size=n, p=[0.12,0.88])
    internet = rng.choice(['DSL','Fiber','No'], size=n, p=[0.4,0.4,0.2])
    contract = rng.choice(['Month-to-month','One year','Two year'], size=n, p=[0.55,0.25,0.2])
    tech = rng.choice([0,1], size=n, p=[0.6,0.4])
    # generate churn probability: higher for short tenure, fiber, month-to-month, high charges, no tech support
    prob = (
        0.35*(tenure<12).astype(float) +
        0.25*(internet=='Fiber').astype(float) +
        0.2*(contract=='Month-to-month').astype(float) +
        0.0007 * (monthly - monthly.mean()) +
        -0.15 * tech
    )
    prob = 1/(1+np.exp(- (prob - prob.mean())))  # squash
    churn = (rng.rand(n) < prob).astype(int)
    df = pd.DataFrame({
        'customerID': [f"C{100000+i}" for i in range(n)],
        'tenure': tenure,
        'MonthlyCharges': monthly,
        'TotalCharges': total,
        'SeniorCitizen': senior,
        'PhoneService': phone,
        'InternetService': internet,
        'Contract': contract,
        'TechSupport': tech,
        'Churn': churn
    })
    return df

df = make_synthetic_telco()
print("Dataset shape:", df.shape)
df.head()

# 2) Basic cleaning & feature engineering
# - Fill/verify TotalCharges
df['TotalCharges'] = df['TotalCharges'].fillna(df['tenure'] * df['MonthlyCharges'])

# - Feature engineering: tenure_buckets, monthly_flag (high_spender), encode categoricals
df['tenure_bucket'] = pd.cut(df['tenure'],
                             bins=[-1,3,12,24,48,72],
                             labels=['0-3','4-12','13-24','25-48','49+'])
df['high_monthly'] = (df['MonthlyCharges'] > df['MonthlyCharges'].median()).astype(int)

# One-hot encode categoricals
cat_cols = ['InternetService','Contract','tenure_bucket']
df_enc = pd.get_dummies(df.drop(columns=['customerID']), columns=cat_cols, drop_first=True)

# 3) Train-test split (holdout) + stratification on Churn
X = df_enc.drop(columns=['Churn'])
y = df_enc['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=42)

# 4) Scaling numeric columns
num_cols = ['tenure','MonthlyCharges','TotalCharges']
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# 5) Cross-validation strategy: Stratified K-Fold (5 folds)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Preprocessing complete. X_train shape:", X_train.shape)
#===========================================================================================
#TASK 2: Train and tune models and compute metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_fscore_support, confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")

# Helper function to evaluate model on test set
def evaluate_model(name, model, X_test, y_test):
    probs = model.predict_proba(X_test)[:,1]
    preds = model.predict(X_test)
    roc_auc = roc_auc_score(y_test, probs)
    ap = average_precision_score(y_test, probs)  # AUC-PR (average precision)
    p, r, f, _ = precision_recall_fscore_support(y_test, preds, average='binary', zero_division=0)
    cm = confusion_matrix(y_test, preds)
    return dict(name=name, roc_auc=roc_auc, avg_precision=ap, precision=p, recall=r, f1=f, confusion_matrix=cm)

results_list = []

# 2A: Logistic Regression baseline
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
results_list.append(evaluate_model("LogisticRegression", lr, X_test, y_test))

# 2B: XGBoost (if available)
try:
    import xgboost as xgb
    xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    # small randomized search for core hyperparams
    param_dist = {
        'n_estimators': [100,200],
        'max_depth': [3,5,7],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.6,0.8,1.0]
    }
    rsearch = RandomizedSearchCV(xgb_clf, param_distributions=param_dist, n_iter=6, cv=cv, scoring='roc_auc', n_jobs=-1, random_state=42)
    rsearch.fit(X_train, y_train)
    best_xgb = rsearch.best_estimator_
    results_list.append(evaluate_model("XGBoost", best_xgb, X_test, y_test))
    print("XGBoost best params:", rsearch.best_params_)
except Exception as e:
    print("XGBoost not available, falling back to RandomForest for advanced model. Error:", e)
    rf = RandomForestClassifier(n_estimators=200, random_state=42)
    rf.fit(X_train, y_train)
    results_list.append(evaluate_model("RandomForest", rf, X_test, y_test))
    best_xgb = rf  # fallback reference

# 2C: LightGBM (if available)
try:
    import lightgbm as lgb
    lgb_clf = lgb.LGBMClassifier(random_state=42)
    param_dist_lgb = {
        'n_estimators': [100,200],
        'num_leaves': [31,50,80],
        'learning_rate': [0.01,0.05,0.1],
        'subsample': [0.6,0.8,1.0]
    }
    rsearch_lgb = RandomizedSearchCV(lgb_clf, param_distributions=param_dist_lgb, n_iter=6, cv=cv, scoring='roc_auc', n_jobs=-1, random_state=42)
    rsearch_lgb.fit(X_train, y_train)
    best_lgb = rsearch_lgb.best_estimator_
    results_list.append(evaluate_model("LightGBM", best_lgb, X_test, y_test))
    print("LightGBM best params:", rsearch_lgb.best_params_)
except Exception as e:
    print("LightGBM not available, skipping. Error:", e)

# 2D: Collect results and print table
import pandas as pd
metrics_df = pd.DataFrame(results_list)
metrics_df = metrics_df[['name','roc_auc','avg_precision','precision','recall','f1']]
print(metrics_df.to_string(index=False))
#===========================================================================================================================
#TASK 3: SHAP global analysis
try:
    import shap
    # Use the selected best model (prefer XGBoost > LightGBM > RandomForest)
    final_model = None
    for candidate in ['XGBoost','LightGBM','RandomForest','LogisticRegression']:
        if candidate in [r['name'] for r in results_list]:
            # find the corresponding trained estimator
            for r in results_list:
                if r['name'] == candidate:
                    # We must pick the trained estimator variable from above. The code above
                    # assigned best_xgb, best_lgb, rf or lr accordingly. To be safe, reconstruct:
                    pass
    # For clarity, we assume best_xgb or best_lgb or rf (as named when training) exists in scope:
    try:
        model_for_shap = best_xgb
    except NameError:
        try:
            model_for_shap = best_lgb
        except NameError:
            model_for_shap = rf  # fallback
    # Create TreeExplainer for tree models
    explainer = shap.TreeExplainer(model_for_shap)
    shap_values = explainer.shap_values(X_train)  # use training or test set
    # Global summary (produce plots when running interactively)
    shap.summary_plot(shap_values, X_train, show=True)
    shap.summary_plot(shap_values, X_train, plot_type="bar", show=True)
except Exception as e:
    print("SHAP not available or failed: ", e)
    print("Install shap: pip install shap  (then re-run this cell)")
# ============================================================
# TASK 4 — Local SHAP Explanations (fixed + robust version)
import shap
import numpy as np

# 1. Make sure SHAP is installed
try:
    shap.__version__
except:
    raise ImportError("Run: pip install shap")

# 2. Ensure LightGBM model is used
final_model = best_lgb   # replace with your lightgbm variable name

# 3. Recompute SHAP explanation safely
explainer = shap.TreeExplainer(final_model)
raw_shap_values = explainer.shap_values(X_test)

# ---- FIX: Handle binary output (1 array) or multiclass (list of arrays) ----
if isinstance(raw_shap_values, list):
    shap_values = raw_shap_values[1]   # Class 1 = churn
else:
    shap_values = raw_shap_values      # Already correct (binary)

# 4. Use your selected indices
indices = {
    "predicted_churn": 1058,
    "predicted_not_churn": 1056,
    "borderline": 18
}

# 5. Generate SAFE Local Plots + Text Outputs
local_outputs = {}

for label, idx in indices.items():
    print(f"\n===== Local SHAP Explanation for: {label} (Index {idx}) =====")

    # Raw contribution values
    vals = shap_values[idx]
    feature_contribs = sorted(
        list(zip(X_test.columns, vals)),
        key=lambda x: abs(x[1]),
        reverse=True
    )

    # Save text explanation
    local_outputs[label] = feature_contribs[:10]

    print("\nTop 10 Feature Contributions:")
    for f, v in feature_contribs[:10]:
        print(f"{f:25s}  SHAP = {v:.4f}")

    # Plot waterfall
    try:
        shap.plots.waterfall(
            shap.Explanation(
                values=vals,
                base_values=explainer.expected_value,
                data=X_test.iloc[idx],
                feature_names=X_test.columns
            ),
            max_display=10
        )
    except Exception as e:
        print("Plot failed:", e)

local_outputs
#====================================================================================================
#Task5: Synthesize global and local SHAP findings into a concise strategic recommendations document for the retention team.
'''Prioritize converting month-to-month customers into 1-2 year contracts. (Strongest global SHAP churn driver.)

Implement proactive onboarding for customers with tenure < 3 months. (High SHAP influence from low tenure.)

Provide pricing relief or bundles for high MonthlyCharges customers. (MonthlyCharges consistently contributes positively to churn.)

Strengthen support for Fiber customers. (Fiber shows higher SHAP churn contribution.)

Deploy TechSupport-based retention initiatives. (Lack of TechSupport pushes churn in SHAP.)

Use local SHAP for personalized retention messaging:

Show customers exactly which issues affect their risk.
Tailor offers to the specific local SHAP drivers.'''
#=========================================================================
def textual_shap_local(idx, shap_values, X_test, model):
    """
    Generate clean textual SHAP explanations for a single sample.
    idx      = index in X_test
    shap_values = array of SHAP values for class 1
    model    = trained LightGBM or RandomForest model
    """

    row_vals = shap_values[idx]
    row = X_test.iloc[idx]

    feature_importance = sorted(
        zip(X_test.columns, row_vals),
        key=lambda x: abs(x[1]),
        reverse=True
    )

    top_pos = [(f, v) for f, v in feature_importance if v > 0][:6]
    top_neg = [(f, v) for f, v in feature_importance if v < 0][:6]

    prob = model.predict_proba([row])[0,1]

    lines = []
    lines.append(f"Index: {idx}, Predicted probability: {prob:.3f}")
    lines.append("Top positive SHAP contributors:")
    for f, v in top_pos:
        lines.append(f"  + {f}: SHAP {v:.4f}")

    lines.append("Top negative SHAP contributors:")
    for f, v in top_neg:
        lines.append(f"  - {f}: SHAP {v:.4f}")

    return "\n".join(lines)
#=========================================================================
print(textual_shap_local(1058, shap_values, X_test, final_model))
print()
print(textual_shap_local(1056, shap_values, X_test, final_model))
print()
print(textual_shap_local(18, shap_values, X_test, final_model))

